In [1]:
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import check_random_state
import sklearn
from sklearn.metrics import roc_auc_score
import random
random.seed(0)

In [2]:
def getMNIST():
    t0 = time.time()
    train_samples = 60000
    # Load data from https://www.openml.org/d/554
    X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
    X = X.reshape((X.shape[0], -1))
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=train_samples, test_size=10000)
    scaler = StandardScaler()
    #scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    print("==================")
    X_train=list(X_train)
    X_test=list(X_test)
    print(len(X_train))
    print(len(X_test))
    print("==================")
    
    return X_train, y_train, X_test, y_test

In [3]:
def make_38_Subset(X_train, y_train, X_test, y_test):
    templist = []
    for tup in zip(X_train, y_train):
        if(tup[1]=='8' or tup[1]=='3'):
            templist.append(list(tup))

    testList = []
    for tup in zip(X_test, y_test):
        if(tup[1]=='8' or tup[1]=='3'):
            testList.append(list(tup))

    print("3/8 Training set size = "+str(len(templist)))
    print("3/8 Test set size = "+str(len(testList)))
    
    X_test=[]
    y_test=[]

    for tup in testList:
        X_test.append(tup[0])
        y_test.append(tup[1])


    seed_size=int(0.1*len(templist))
    seed_list=templist[0:seed_size]
    print("Initial Seed set size = "+str(len(seed_list)))
 
    unlabelled_list=templist[seed_size:]
    print("Unlabelled dataset size = "+str(len(unlabelled_list)))
    
    return seed_list, unlabelled_list, X_test, y_test


In [4]:
def random_instance_generate(seed_set, unlabelled_list, batch_size):
    random_elements=[]
    print("Before Sampling Unlabelled Data Size: "+str(len(unlabelled_list)))
    print("Before Sampling Seed Data Size: "+str(len(seed_set)))
    random.shuffle(unlabelled_list)
    for i in range(batch_size):
        rand_elem = random.choice(unlabelled_list)
        random_elements.append(rand_elem)
    for elem in random_elements:
        seed_set.append(elem)
        ul=[]
    for i in random_elements:
        ul.append(list(i[0]))
        
    new_unlabelled=[]
    for elem in unlabelled_list:
        if(list(elem[0]) in ul):
            continue
        new_unlabelled.append(elem)
    print("After Sampling Unlabelled Data Size : "+str(len(new_unlabelled)))
    print("After Sampling Seed Data Size : "+str(len(seed_set)))
    return new_unlabelled, seed_set

In [5]:
# split unlabeled into del_s (new points to add to S) and points removed from U (a new list)
def transform_u(u, selected):
    modified_u = [] # modified unlabeled
    del_s = [] # new points to add to s
    
    i=0
    for x in u:
        if i in selected:
            del_s.append(x)
        else:
            modified_u.append(x)            
        i=i+1
    return del_s, modified_u

In [6]:
import numpy as np
import numpy.random as npr

def select_random_from_unlabeled(u, batch_size, model=None):
    selected = set(npr.choice(len(u), batch_size, replace=False)) #these will no longer be unlabeled
    return transform_u(u, selected)

In [7]:
from functools import reduce
import operator

def prod(iterable):
    return reduce(operator.mul, iterable, 1)

In [16]:
#compare pairs of the form (id, value)
import functools

#descending order
def compare_by_val(u, v):
    if u[1] < v[1]:
        return 1
    elif u[1] == v[1]:
        return 0
    else:
        return -1

    
# the max value of the predicted probabilities = 1/num_classes
# so if this value is small - it denotes a level of certainty
# maximize these values for the uncertainty sampling
def compute_uncertainty(predicted_probs):
    return prod(predicted_probs)
    
def select_most_uncertain_from_unlabeled(u, batch_size, model):    
    #compute probs
    avp_list = []
    i = 0
    for x, y in u:
        prob_wts = model.predict_proba([x])
        p = compute_uncertainty(prob_wts[0]) # to be maximized
        avp_list.append([i, p])
        i=i+1
    
    #sort
    sorted(avp_list, key=functools.cmp_to_key(compare_by_val))
    
    #select top ones - most uncertain
    selected = []
    i = 0
    for i, p_i in avp_list:
        selected.append(i)
        i=i+1
        if i==batch_size:
            break
        
    return transform_u(u, selected)

In [20]:
def train_model(data, X_test, y_test):
    #print("--------------------Start------------------")
    
    clf = LogisticRegression(C=0.01, penalty='l2', solver='saga', tol=0.1)    
    #print("Training with seed size: " + str(len(data)))
    
    X_train=[]
    y_train=[]
    for i in range(len(data)):
        try:
            X_train.append(list(data[i][0]))
            y_train.append(data[i][1])
        except:
            pass
        
    clf.fit(X_train, y_train)
    
    #print("Accuracy on the Test Set is : "+str(sklearn.metrics.accuracy_score(y_test, clf.predict(X_test))))
    #print("--------------------End------------------")
    return(clf)


In [10]:
print ("Making MNIST data...")
X_train, y_train, X_test, y_test = getMNIST()

Making MNIST data...
60000
10000


In [21]:
print ("Getting MNIST 3/8 classification data...")
s, u, X_test, y_test = make_38_Subset(X_train, y_train, X_test, y_test)

#Define the available selector functions
selector_functions = {
  'usample': select_most_uncertain_from_unlabeled,
  'random': select_random_from_unlabeled
}

results = {}
niters = 3
batch_size = 1000

for selector_function in selector_functions.keys():
    #print ("Results with: {}".format(selector_function))
    
    for i in range(niters):
        # print("|S|_{} = {}, |U|_{} = {}".format(i, len(s), i, len(u)))
        model = train_model(s, X_test, y_test)
        del_s, u = selector_functions[selector_function](u, batch_size, model)    
        s = s + del_s
        
        key = selector_function + ", " + str(i)
        acc_val = str(sklearn.metrics.accuracy_score(y_test, model.predict(X_test)))
        results[key] = acc_val

for key in results.keys():
    print ("{} = {}".format(key, results[key]))

Getting MNIST 3/8 classification data...
3/8 Training set size = 11987
3/8 Test set size = 1979
Initial Seed set size = 1198
Unlabelled dataset size = 10789
usample, 0 = 0.9191510864072764
usample, 1 = 0.9307731177362304
usample, 2 = 0.9363314805457301
random, 0 = 0.9398686205154119
random, 1 = 0.9413845376452754
random, 2 = 0.9464375947448206
